In [61]:
using Pkg
Pkg.activate("/home/fedflorio/master_thesis")

  Activating project at `~/master_thesis`


In [62]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, PyPlot, DelimitedFiles
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
include("/home/fedflorio/master_thesis/Utilities/roc.jl");

In [63]:
A = readdlm("/home/fedflorio/master_thesis/MatrixProductBP.jl/notebooks/karate.txt", Bool)
g = IndexedGraph(A)

nsnaps = 300
separation = 128
T = nsnaps * separation
N = nv(g)
seed = 4

λ_unif = 0.025
ρ_unif = 0.05
λ = zeros(N,N)
for i in CartesianIndices(λ)
    if !iszero(g.A[i])
        # λ[i] = rand()
        λ[i] = λ_unif
    end
end
λ = sparse(λ)
# ρ = rand(N)
ρ = fill(ρ_unif,N)
γ = [i==1 ? 1.0 : 0.0 for i in 1:N]
# γ = 0.1
;

In [64]:
sis = SIS_heterogeneous(λ, ρ, T; γ)
bp_obs = mpbp(sis);

In [65]:
obs_times = collect(range(separation, step=separation, length=nsnaps))
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp_obs, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [66]:
X

34×38401 Matrix{Int64}:
 2  2  2  2  2  2  2  2  2  2  2  2  1  …  1  1  1  2  2  2  2  2  2  2  2  2
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  2  2  2  2  2  2  2  2  2
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  2  2  2  2  2  2  2  1  1  2  2
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     2  2  2  2  2  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  2  2  2  2  2  2  …  1  1  1  2  2  2  2  2  2  2  2  2
 1  1  1  1  1  1  1  1  1  1  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  2     1  1  1  1  2  2  2  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  2  2  2  2  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     2  2  2  2  2  2  2  2  2  2  2  2
 1  1  1  1  1  1  1  2  2  1  2  2  2  …  2  2  2  2  2  2  2  2  2  2  2  2
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  

In [ ]:
λinit = 0.1
ρinit = 0.1

A_complete = ones(N,N) - I
g_complete = IndexedGraph(A_complete)
λ_complete = sparse(λinit.*A_complete)
ρ_complete = fill(ρinit, N)

sis_inf = SIS_heterogeneous(g_complete, λ_complete, ρ_complete, T; γ, ϕ=deepcopy(bp_obs.ϕ))
bp_inf = mpbp(sis_inf);

In [ ]:
svd_trunc = TruncBond(3)
maxiter = 40

for iter in 1:maxiter
    iters, cb = inference_parameters!(bp_inf, method=31, maxiter=1, λstep=0.01, ρstep=0.01, logpriorder=(x)->0.0);

    xplot, yplot, auc = roc(cb.data[end].λ, λ)
    @show auc

    close("all")
    plot(xplot,yplot)
    gcf()
end